### Pre-installation of Dependencies

To run the player and ball tracking code, make sure you have the following libraries installed:

```bash
pip install opencv-python-headless
pip install torch
pip install ultralytics
pip install numpy
```

Ensure that you have the YOLOv8 model (`yolov8n.pt`) downloaded and available in the working directory.

This code requires Python 3.9 or 3.10 for compatibility with the YOLOv8 model and other dependencies.

In [ ]:
import cv2
import torch
from ultralytics import YOLO
import time
import numpy as np

# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Make sure to have the pretrained yolov8n.pt file

# Initialize video capture for live video (default camera)
capture = cv2.VideoCapture(0)

# Variables for tracking
prev_ball_position = None  # Previous position of the ball
ball_speed_threshold = 45  # Speed threshold for ball tracking

if not capture.isOpened():
    print("Error: Could not open the video camera.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = capture.read()
    if not ret:
        print("Error: Failed to capture frame from camera.")
        break

    # Run YOLOv8 model on frame
    results = model(frame)
    detections = results[0].boxes.data.cpu().numpy() if results else []

    # Current positions
    current_player_positions = []
    current_ball_position = None

    for detection in detections:
        x1, y1, x2, y2, score, class_id = detection[:6]
        if class_id == 0:  # Assuming class 0 is person
            label = f"Person: {score:.2f}"
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            current_player_positions.append(((int(x1) + int(x2)) // 2, (int(y1) + int(y2)) // 2))  # Center of the box
        elif class_id == 32:  # Assuming class 32 is ball
            label = f"Ball: {score:.2f}"
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            current_ball_position = ((int(x1) + int(x2)) // 2, (int(y1) + int(y2)) // 2)  # Center of the box

    # Calculate ball speed if previous position exists
    if prev_ball_position is not None and current_ball_position is not None:
        ball_speed = np.linalg.norm(np.array(current_ball_position) - np.array(prev_ball_position))
        cv2.putText(frame, f'Ball Speed: {ball_speed:.2f}', (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

    # Update previous ball position for the next frame
    prev_ball_position = current_ball_position

    # Display the number of players detected on the frame
    cv2.rectangle(frame, (10, 10), (310, 60), (255, 255, 255), -1)
    cv2.putText(frame, f'Total Players: {len(current_player_positions)}', (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Display the frame
    cv2.imshow('Player and Ball Tracking - YOLOv8', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
capture.release()
cv2.destroyAllWindows()
